In [1]:
#general imports
import os
import platform
import re
import logging
from pprint import pprint

#spikeinterface imports
import spikeinterface
import spikeinterface.sorters as ss
import spikeinterface.full as si

#local imports
from file_selector import main as file_selector_main
#from AxonReconPipeline.src.archive.extract_raw_assay_data import main as extract_raw_assay_data_main
from generate_recording_and_sorting_objects import main as generate_recording_and_sorting_objects
from axon_trace_classes import axon_trace_objects
from reconstruct_axons import reconstruct_axon_trace_objects
import mea_processing_library as MPL
import helper_functions as helper
import spike_sorting as hp #hornauerp, https://github.com/hornauerp/axon_tracking/blob/main/axon_tracking/spike_sorting.py
import template_extraction as te_hp

#Logger Setup
#Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create handlers
stream_handler = logging.StreamHandler()  # logs to console
#file_handler = logging.FileHandler('file.log')  # logs to a file

# Set level of handlers
stream_handler.setLevel(logging.DEBUG)
#file_handler.setLevel(logging.ERROR)

# Add handlers to the logger
logger.addHandler(stream_handler)
#logger.addHandler(file_handler)

# Create formatters and add it to handlers
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s - %(module)s.%(funcName)s')
stream_handler.setFormatter(formatter)

{'detect_threshold': 0.01,
 'detection_type': 'relative',
 'distance_exp': 2,
 'edge_dist_amp_ratio': 0.3,
 'init_amp_peak_ratio': 0.2,
 'init_delay': 0,
 'kurt_threshold': 0.1,
 'mad_threshold': 8,
 'max_distance_for_edge': 300,
 'max_distance_to_init': 200,
 'max_peak_latency_for_splitting': 0.5,
 'min_outlier_tracking_error': 50,
 'min_path_length': 100,
 'min_path_points': 5,
 'min_points_after_branching': 3,
 'min_selected_points': 30,
 'n_neighbors': 3,
 'neighbor_radius': 100,
 'peak_std_distance': 30,
 'peak_std_threshold': 0.8,
 'r2_threshold': 0.8,
 'r2_threshold_for_outliers': 0.98,
 'remove_isolated': True,
 'split_paths': True,
 'theilsen_maxiter': 10000,
 'upsample': 5}


In [2]:
#import all functions in main.py
from main import *

debug_mode = True
#testing Full Activity Scan and Network Scan...
pre_selected_folders = [        
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/ActivityScan/000023",
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/Network/000024",
    "/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026",
    #testing continuity test with non-continuous data
    #"/mnt/disk20tb/PrimaryNeuronData/Maxone/SPTAN1_1/230725/16657/ActivityScan",

    #new comparison 09Feb24
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M05506/AxonTracking/000003"
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M07038/AxonTracking"
    ]
continuous_h5_dirs = select_folders_and_get_continuous_h5_dirs(pre_selected_folders = pre_selected_folders, debug_mode = debug_mode)
pprint(continuous_h5_dirs)

2024-02-17 15:36:58,981 - INFO - Selecting folders to process, checking for continuity, and extracting the .h5 file path: - main.select_folders_and_get_continuous_h5_dirs
2024-02-17 15:36:58,982 - INFO - Selected folders: ['/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026'] - main.select_folders_and_get_continuous_h5_dirs
2024-02-17 15:36:58,983 - INFO - Extracting .h5 file paths from directories: - mea_processing_library.extract_raw_h5_filepaths



*** DEBUG MODE ENABLED in file_selector.py ***
/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026

Folders selected for analysis:


2024-02-17 15:36:59,770 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:36:59,771 - INFO - Testing continuity of /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026/data.raw.h5: - mea_processing_library.test_continuity
2024-02-17 15:36:59,771 - INFO - Counting wells and recordings in /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026/data.raw.h5: - mea_processing_library.count_wells_and_recs
2024-02-17 15:36:59,772 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:36:59,773 - INFO - Scan Type: AxonTracking - mea_processing_library.count_wells_and_recs
2024-02-17 15:37:00,302 - INFO - MaxTwo Detected. - mea_processing_library.count_wells_and_recs
2024-02-17 15:37:13,281 - INFO - Stream ID: w

Data for /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026/data.raw.h5 is continuous.
{0: {'chipID': 'M06844',
     'date': '240118',
     'h5_file_path': '/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026/data.raw.h5',
     'runID': '000026',
     'scanType': 'AxonTracking'}}


In [3]:
#Set working directory for following cells to work properly (if running in Jupyter Notebook)
#This directory should be the root of the git repository
working_dir = os.getcwd()
if working_dir not in '/mnt/disk15tb/adam/git_workspace/MEA_Analysis':
    os.chdir('/mnt/disk15tb/adam/git_workspace/MEA_Analysis')
    working_dir = os.getcwd()
print(working_dir)

/mnt/disk15tb/adam/git_workspace/MEA_Analysis


In [4]:
#Set up matplotlib to work properly in Jupyter Notebook
%matplotlib inline

In [5]:

#continuous_h5_dirs = 

#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan only needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
recordings_dir = './AxonReconPipeline/data/temp_data/recordings'
#only get sortings for well001
stream_select = 1
##2. Generate multirecording objects for each stream for spikesorting and waveform extraction steps
multi_recs_by_scan, common_el_by_scan = generate_multirecordings_by_stream_by_scan(continuous_h5_dirs,
                                                                                    allowed_scan_types,
                                                                                    recordings_dir,
                                                                                    stream_select = stream_select,
                                                                                    )

2024-02-17 15:37:33,289 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:37:35,153 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:37:35,290 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0000 - spike_sorting.concatenate_recording_slices
2024-02-17 15:37:35,544 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0001 - spike_sorting.concatenate_recording_slices
2024-02-17 15:37:35,693 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0002 - spike_sorting.concatenate_recording_slices
2024-02-17 15:37:35,852 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/d

In [20]:

#Plot common electrodes
recording_dir = './AxonReconPipeline/data/temp_data/recordings'
#note this automatically saves the plot to the recording_dir for the date/chip/scan/run/stream 
widgets_by_scan = plot_electrodes(multi_recs_by_scan, recording_dir=recording_dir, auto_plot = False)
#plt.tight_layoutplot_progressive_recording_channel_exclusion_plots
#plt.show()

2024-02-18 12:57:56,722 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details


KeyboardInterrupt: 

In [7]:
import numpy as np
import spike_sorting as hp

#generate progressive channel exclusion plots
def plot_progressive_recording_channel_exclusion_plots(rec_path, 
                                 stream_id, 
                                 recording_dir = './AxonReconPipeline/data/temp_data/recordings/',
                                 #stream_break = None
                                 ):
    """
    Function that centers and concatenates the recordings of an axon scan for all common electrodes. 

    Arguments
    ----------
    rec_path: str
        Path to the axon scan file.
    stream_id: str
        Well ID in the format "well***"; Well 1 would be "well001", Well 20 would be "well020"
        
    Returns
    ----------
    multirecording: ConcatenatedRecordingSlice
        Concatenated recording across common electrodes (spikeinterface object)
    """
    rec_names, common_el = hp.find_common_electrodes(rec_path, stream_id)
    recording_details = MPL.extract_recording_details(rec_path)
    date = recording_details[0]['date']
    chip_id = recording_details[0]['chipID']
    scanType = recording_details[0]['scanType']
    run_id = recording_details[0]['runID']
    recording_dir_by_stream =recording_dir+f'/{date}/{chip_id}/{scanType}/{run_id}'
    
    rec_list = []
    recording_dir_by_stream = os.path.join(recording_dir_by_stream, stream_id)
    
        
    for rec_name in rec_names: 
        #rec_name = 'rec' + '%0*d' % (4, r)
        rec = si.MaxwellRecordingExtractor(rec_path, stream_id=stream_id, rec_name=rec_name)
                
        ch_id = rec.get_property("contact_vector")['device_channel_indices']
        rec_el = rec.get_property("contact_vector")["electrode"]
        
        chan_idx = [np.where(rec_el == el)[0][0] for el in common_el]
        sel_channels = rec.get_channel_ids()[chan_idx]
        #chunk_size = np.min([10000, rec.get_num_samples()]) - 100 #Fallback for ultra short recordings (too little activity)
        #rec_centered = si.center(rec,chunk_size=chunk_size)
        #rec_list.append(rec_centered)
        rec_wrap = [[rec]]
        #recording_dir = recording_dir
        #note this automatically saves the plot to the recording_dir for the date/chip/scan/run/stream 
        widgets_by_scan = plot_electrodes(rec_wrap, recording_dir=recording_dir, auto_plot = False, name = rec_name)

        #rec_list.append(rec.channel_slice(sel_channels, renamed_channel_ids=list(range(len(chan_idx)))))
        #multirecording = si.concatenate_recordings(rec_list)
        #widgets_by_scan = plot_electrodes([[multirecording]], recording_dir=recording_dir, auto_plot = False, name = f"prog_{rec_name}")
    
    

    # if not os.path.exists(recording_dir_by_stream):
    #     os.makedirs(recording_dir_by_stream)
    # save_kwargs = dict(n_jobs=12)
    # multirecording.save(folder = recording_dir_by_stream, overwrite = True, verbose = True, **save_kwargs)

    # return multirecording, common_el
        
#plot_progressive_recording_channel_exclusion_plots(continuous_h5_dirs[0]['h5_file_path'], "well001", recording_dir = recording_dir)        

In [13]:
#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan only needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
recordings_dir = './AxonReconPipeline/data/temp_data/recordings'
sortings_dir = './AxonReconPipeline/data/temp_data/sortings'
#only get sortings for well001
stream_select = 1
##3. Extract sorting objects per stream from sliced and cocatenated recordings (most/all fixed electrodes during axontracking scan)
sorting_lists_by_scan = get_list_of_sortings_by_scan(continuous_h5_dirs, 
                                                     recordings_dir = recordings_dir,
                                                     sortings_dir = sortings_dir, 
                                                     allowed_scan_types = allowed_scan_types,
                                                     stream_select = stream_select,)

2024-02-17 15:52:02,800 - INFO - Loading and merging the recordings as needed from the .h5 files. Generating spike sorting objects and merging as needed. - main.get_list_of_sortings_by_scan
2024-02-17 15:52:02,802 - INFO - Allowed scan types: ['AxonTracking'] - main.get_list_of_sortings_by_scan
2024-02-17 15:52:02,803 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
Sorting recordings:   0%|          | 0/1 [00:00<?, ?it/s]2024-02-17 15:52:14,580 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:52:14,691 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0000 - spike_sorting.concatenate_recording_slices
2024-02-17 15:52:14,766 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec000

DURATION: 349.76 s -- NUM. CHANNELS: 354
Starting container
Installing spikeinterface==0.99.1 in spikeinterface/kilosort2_5-compiled-base:latest
Running kilosort2_5 sorter inside spikeinterface/kilosort2_5-compiled-base:latest
Stopping container


Spike sorting elapsed time 411.39704179763794 s


Sorting recordings: 100%|██████████| 1/1 [07:06<00:00, 426.94s/it]


In [ ]:
# import matplotlib.pyplot as plt
# import spikeinterface.extractors as se

# # Assuming sorting_lists_by_scan[0][0] is a SortingExtractor instance
# sorting = sorting_lists_by_scan[0][0]

# # Get unit ids
# unit_ids = sorting.get_unit_ids()

# # Get unit locations
# unit_locations = [sorting.get_unit_property(unit_id, 'location') for unit_id in unit_ids]

# # Unpack locations
# x_locs = [loc[0] for loc in unit_locations]
# y_locs = [loc[1] for loc in unit_locations]

# # Create plot
# plt.scatter(x_locs, y_locs)
# plt.title('Unit Locations')
# plt.xlabel('X Location')
# plt.ylabel('Y Location')
# plt.show()

In [14]:
import template_extraction as te_hp
import tqdm

def extract_waveform_from_segments(h5_file_path, stream_id, segment_sorting, sel_unit_id, save_root, peak_cutout=2, align_cutout=True, upsample=2, rm_outliers=True, n_jobs=16, n_neighbors=10, overwrite_wf=False, overwrite_tmp = False):
    #full_path = segment_sorting._kwargs['recording_list'][0]._parent_recording._kwargs['recording']._kwargs['file_path']
    try:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._kwargs['parent_recording']._kwargs['recording']._kwargs['file_path']
    except:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._parent_recording._kwargs['folder_path']
        #segment_sorting._kwargs['recording_list'][0]._kwargs['parent_recording']._kwargs['folder_path']
        #get parent folder instead of file path
        full_path = os.path.dirname(full_path)
    
    if save_root is None:
        #save_root = os.path.dirname(full_path) 
        recording_details = MPL.extract_recording_details(full_path)
        date = recording_details[0]['date']
        chip_id = recording_details[0]['chipID']
        scanType = recording_details[0]['scanType']
        run_id = recording_details[0]['runID']
        save_root = f'./AxonReconPipeline/data/temp_data/waveforms/{date}/{chip_id}/{scanType}/{run_id}/{stream_id}'
    waveforms = te_hp.extract_waveforms(h5_file_path, segment_sorting, stream_id, save_root, n_jobs, overwrite_wf)
    return waveforms

def extract_all_waveforms(h5_file_path, stream_id, segment_sorting, pos, te_params, save_root=None):
    sel_unit_ids = segment_sorting.get_unit_ids()
    try:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._kwargs['parent_recording']._kwargs['recording']._kwargs['file_path']
    except:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._parent_recording._kwargs['folder_path']
        #segment_sorting._kwargs['recording_list'][0]._kwargs['parent_recording']._kwargs['folder_path']
        #get parent folder instead of file path
        full_path = os.path.dirname(full_path)
    
    if save_root is None:
        #save_root = os.path.dirname(full_path) 
        recording_details = MPL.extract_recording_details(full_path)
        date = recording_details[0]['date']
        chip_id = recording_details[0]['chipID']
        scanType = recording_details[0]['scanType']
        run_id = recording_details[0]['runID']
        save_root = f'./AxonReconPipeline/data/temp_data/waveforms/{date}/{chip_id}/{scanType}/{run_id}/{stream_id}'
    waveform_save_path = os.path.join(save_root, 'waveforms')
    if not os.path.exists(waveform_save_path):
        os.makedirs(waveform_save_path)
        
    for sel_unit_id in sel_unit_ids: 
        #template_save_file = os.path.join(template_save_path, str(sel_unit_id) + '.npy')
        break
        
    if te_params['overwrite_tmp']:
        try:
            extract_waveform_from_segments(h5_file_path, stream_id, segment_sorting, sel_unit_id, save_root, **te_params)
            #grid = convert_to_grid(template_matrix, pos)
            #np.save(template_save_file, merged_template)
        except Exception as e:
            print(f'{e}')

def extract_waveforms_from_sorting_dict(sorting_list, h5_file_path, qc_params={}, te_params={}, stream_break = None):
    #rec_list = list(sorting_dict.keys())
    
    rec_list = h5_file_path
    if isinstance(h5_file_path, str):
        rec_list = [h5_file_path]    

    for rec_path in rec_list:
        #sorting_list = sorting_dict[rec_path]
        #sorting_list = sorting_dict
        #sorting_list.sort()

        for sorting in sorting_list:
        #for sorting_path in sorting_list:
            #sorting = si.KiloSortSortingExtractor(sorting_path)
            sorting_path = sorting._kwargs['folder_path']
            # #Temporary debug
            # #if 'Network' in stream_id, replace with 'AxonTracking'
            # if 'Network' in sorting_path:
            #     sorting_path = sorting_path.replace('Network', 'AxonTracking')
            stream_id = [p for p in sorting_path.split('/') if p.startswith('well')][0] #Find out which well this belongs to
            
            # print(stream_id)
            # #debug:
            # stream_number = int(stream_id[-1])
            # if stream_number > 0:
            #     continue
            # #debug end
            
            #rec_names, common_el, pos = ss.find_common_electrodes(rec_path, stream_id)
            multirecording, pos = hp.concatenate_recording_slices(rec_path, stream_id)

            #duration = int(h5['assay']['inputs']['record_time'][0].decode('UTF-8')) * n_recs #In case we want to use firing rate as criterion
            cleaned_sorting = te_hp.select_units(sorting, **qc_params)
            cleaned_sorting = si.remove_excess_spikes(cleaned_sorting, multirecording) #Relevant if last spike time == recording_length
            cleaned_sorting.register_recording(multirecording)
            segment_sorting = si.SplitSegmentSorting(cleaned_sorting, multirecording)
            extract_all_waveforms(h5_file_path, stream_id, segment_sorting, pos, te_params, save_root=None)

            if stream_break is not None:
                formatted = 'well{:03}'.format(stream_break)
                if stream_id == stream_break:
                    break

te_params = dict()
te_params['align_cutout'] = True #Align waveforms by max waveform peak
te_params['upsample'] = 2 #Factor by which to upsample waveforms
te_params['rm_outliers'] = True #Check if outliers should be removed
te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
te_params['n_neighbors'] = 10 #Number of neighbors for outlier detection
te_params['peak_cutout'] = 2 #Looking for peak +- this value around the expected peak (removing minor offsets)
te_params['overwrite_wf'] = False #Flag if waveform extraction should be repeated (e.g. different cutouts)
te_params['overwrite_tmp'] = True #Flag if templates should be recalculated if already existing

qc_params = dict()
qc_params['min_n_spikes'] = 500 #Minimum number of spikes to be detected for a unit for template extraction to take place
qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort

for i, sorting_list_by_stream in enumerate(sorting_lists_by_scan):
    extract_waveforms_from_sorting_dict(sorting_list_by_stream, continuous_h5_dirs[i]['h5_file_path'], qc_params=qc_params, te_params=te_params, stream_break = None)

2024-02-17 15:59:21,553 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 15:59:21,649 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0000 - spike_sorting.concatenate_recording_slices
2024-02-17 15:59:21,724 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0001 - spike_sorting.concatenate_recording_slices
2024-02-17 15:59:21,793 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0002 - spike_sorting.concatenate_recording_slices
2024-02-17 15:59:21,879 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0003 - spike_sorting.concatenate_recording_slices
2024-02-17 15:

extract waveforms memmap multi buffer:   0%|          | 0/9 [00:00<?, ?it/s]

2024-02-17 16:00:09,089 - INFO - Extracting waveforms to /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240118/M06844/AxonTracking/000026/well001/waveforms/seg1 - template_extraction.extract_waveforms


extract waveforms memmap multi buffer:   0%|          | 0/8 [00:00<?, ?it/s]

2024-02-17 16:00:27,699 - INFO - Extracting waveforms to /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240118/M06844/AxonTracking/000026/well001/waveforms/seg2 - template_extraction.extract_waveforms


...

extract waveforms memmap multi buffer:   0%|          | 0/9 [00:00<?, ?it/s]

2024-02-17 16:13:24,004 - INFO - Extracting waveforms to /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240118/M06844/AxonTracking/000026/well001/waveforms/seg38 - template_extraction.extract_waveforms


extract waveforms memmap multi buffer:   0%|          | 0/10 [00:00<?, ?it/s]

2024-02-17 16:13:40,671 - INFO - Extracting waveforms to /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240118/M06844/AxonTracking/000026/well001/waveforms/seg39 - template_extraction.extract_waveforms


extract waveforms memmap multi buffer:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan only needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
templates_dir='./AxonReconPipeline/data/temp_data/templates'  
##3. Extract Templates
#Extract recording details from the .h5 file path
#only get tempaltes for well000
stream_select = 1
just_load_waveforms = True
templates_by_scan = get_merged_templates_by_unit_by_scan(continuous_h5_dirs, 
                                                         sorting_lists_by_scan,
                                                         allowed_scan_types=allowed_scan_types,
                                                         templates_dir = templates_dir,
                                                         stream_select = stream_select, 
                                                         just_load_waveforms = just_load_waveforms)

2024-02-17 16:13:55,806 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-02-17 16:13:56,094 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0000 - spike_sorting.concatenate_recording_slices
2024-02-17 16:13:56,186 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0001 - spike_sorting.concatenate_recording_slices
2024-02-17 16:13:56,296 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0002 - spike_sorting.concatenate_recording_slices
2024-02-17 16:13:56,412 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/data/temp_data/recordings/240118/M06844/AxonTracking/000026/well001/rec0003 - spike_sorting.concatenate_recording_slices
2024-02-17 16:

In [22]:


from IPython.core.debugger import set_trace
from pathlib import Path

### Initiate Axon trace algo
import numpy as np
import matplotlib.pylab as plt
import MEAutility as mu
from scipy.signal import resample_poly
from scipy.stats import kurtosis, linregress
from matplotlib import gridspec
from scipy import io
import numpy as np
import networkx as nx
from pathlib import Path
import sys
from pprint import pprint
from axon_velocity import *
from axon_velocity.models import load_cell
from axon_velocity.evaluation import *
import os
from pathlib import Path
import pickle
import re
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from probeinterface import plotting as plotting_probe
import json

import concurrent.futures
from tqdm import tqdm
from multiprocessing import Manager

#for testing
import numpy as np
from shapely.geometry import Polygon
import itertools
from shapely.geometry import Point
from shapely.geometry import Polygon
from collections import Counter
import math
from collections import defaultdict

#Define Algorithm Params
params = get_default_graph_velocity_params()
# change params (fig6 Buccino axon_velocity paper example)
params['detect_threshold'] = 0.01
params['kurt_threshold'] = 0.1
params['peak_std_threshold'] = 0.8
params['upsample'] = 5
params['neighbor_radius'] = 100
params['r2_threshold'] = 0.8

# AW invention. I dont remember why I did this. 4Jan2023
#params["theilsen_maxiter"] = 10000
#pprint(params)
#

stream_select = 1
templates_dir='./AxonReconPipeline/data/temp_data/templates'
for i, continuous_h5_dir in continuous_h5_dirs.items():
    recording_details = MPL.extract_recording_details(continuous_h5_dir['h5_file_path'])
    date = recording_details[0]['date']
    chip_id = recording_details[0]['chipID']
    scanType = recording_details[0]['scanType']
    run_id = recording_details[0]['runID']
    h5 = h5py.File(continuous_h5_dir['h5_file_path'], 'r')
    stream_ids = list(h5['wells'].keys())
    for stream_id in stream_ids:
        if stream_select is not None:
            if stream_id != f'well{stream_select:03}':
                continue
            else:
                print(f'Processing stream {stream_id}')
                break

    temp_dir = os.path.dirname(templates_dir)
    sorting_dir = Path(temp_dir) /'sortings'/ date / chip_id / scanType / run_id / stream_id / 'sorter_output'
    sorting = ss.Kilosort2_5Sorter._get_result_from_folder(sorting_dir)
    recon_dir = Path(temp_dir) /'reconstructions'/ date / chip_id / scanType / run_id / stream_id  
    unit_ids = sorting.get_unit_ids()
    for unit_id in unit_ids:
        merged_template_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}.npy'
        merged_channel_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}_channels.npy'
        try:
            merged_template = np.load(merged_template_dir)
            merged_channel_loc = np.load(merged_channel_dir)
        except:
            #print(f'No merged template found for {unit_id}')
            continue

        # #TESTING A THEORY
        # #look at each column in merged template, find the one with the highest avg value. 
        # #Then, considering the locations of other channels, create the largest possible rectangle around the channel with the highest avg value such that there's a channel at each corner of the rectangle. 
        # #Then cut out the rectangle from the template by removing channels and columns that are not in the rectangle.
        # # Calculate column averages

        # def calculate_column_averages(merged_template):
        #     # Calculate the average of each column in merged_template
        #     return np.mean(merged_template, axis=0)

        # def find_max_average_column(averages):
        #     # Find the index of the column with the highest average
        #     return np.argmax(averages)#, axis=0)

       

        # def is_rectangle(polygon):
        #     if len(polygon.exterior.coords) != 5:  # 4 corners and 1 more because the polygon is closed
        #         return False

        #     # Calculate the distances between each pair of points
        #     points = list(polygon.exterior.coords)
        #     distances = []
        #     for i in range(4):
        #         for j in range(i + 1, 4):
        #             dx = points[i][0] - points[j][0]
        #             dy = points[i][1] - points[j][1]
        #             distances.append(math.sqrt(dx * dx + dy * dy))

        #     # If the polygon is a rectangle, there should be two distinct distances
        #     return len(set(distances)) <= 2

        # def find_largest_rectangle(channel_locations, max_average_channel):
        #     # import numpy as np
        #     # from shapely.geometry import Polygon, Point
        #     # import itertools
        #     # max_area = 0
        #     # rectangle_channels = None
        #     channel_locations = channel_locations[0]


        #     # Assuming channel_locations is a numpy array of numpy arrays
        #     n = len(channel_locations)
        #     r = 4
        #     num_combinations = math.comb(n, r)
        #     print(f'Total Number of possible combinations: {num_combinations}')

        #     # Group channels by x and y coordinates
        #     channels_by_x = defaultdict(list)
        #     channels_by_y = defaultdict(list)
        #     for i, (x, y) in enumerate(channel_locations):
        #         channels_by_x[x].append(i)
        #         channels_by_y[y].append(i)

        #     # Generate combinations of channels where 2 channels share an x coordinate and 2 channels share a y coordinate
        #     combinations = []
        #     for x_channels in channels_by_x.values():
        #         print(f'x_channels: {x_channels}')
        #         ##
        #         if channel_locations[max_average_channel] in channel_locations[x_channels]: continue
        #         ##
        #         max_y_distance = max(channel_locations[x_channels], key=lambda x: x[1])[1] - min(channel_locations[x_channels], key=lambda x: x[1])[1]
        #         if max_y_distance < 20: continue
        #         ##
        #         # Sort channel_locations[x_channels] by the y-coordinate
        #         sorted_locations = sorted(channel_locations[x_channels], key=lambda x: x[1])

        #         # Initialize the list of updated locations with the first location
        #         updated_locations = [sorted_locations[0]]

        #         # Iterate over the sorted locations
        #         for i in range(1, len(sorted_locations)):
        #             # If the current location is spaced at least 10 microns apart from the last location in updated_locations, add it to updated_locations
        #             if sorted_locations[i][1] - updated_locations[-1][1] >= 20:
        #                 updated_locations.append(sorted_locations[i])

        #         # Update channel_locations[x_channels] with the updated locations
        #         channel_locations_x = updated_locations
        #         ##
        #         for y_channels in channels_by_y.values():
        #             print(f'y_channels: {y_channels}')
        #             ##
        #             if channel_locations[max_average_channel] in channel_locations[y_channels]: continue
        #             ##
        #             max_x_distance = max(channel_locations[y_channels], key=lambda x: x[0])[0] - min(channel_locations[y_channels], key=lambda x: x[0])[0]
        #             if max_x_distance < 20: continue
        #             ##
        #             # Sort channel_locations[x_channels] by the y-coordinate
        #             sorted_locations = sorted(channel_locations[y_channels], key=lambda x: x[0])
        #             ##
        #             # Initialize the list of updated locations with the first location
        #             updated_locations = [sorted_locations[0]]

        #             # Iterate over the sorted locations
        #             for i in range(1, len(sorted_locations)):
        #                 # If the current location is spaced at least 10 microns apart from the last location in updated_locations, add it to updated_locations
        #                 if sorted_locations[i][0] - updated_locations[-1][0] >= 20:
        #                     updated_locations.append(sorted_locations[i])

        #             # Update channel_locations[x_channels] with the updated locations
        #             channel_locations_y = updated_locations

                    
        #             # in channel_locations[x_channels] the maximum distance in the y coordinate must be at least 10 um
        #             # in channel_locations[y_channels] the maximum distance in the x coordinate must be at least 10 um
        #             # Calculate the maximum distance in the y coordinate for x_channels
                    
        #             # Calculate the maximum distance in the x coordinate for y_channels
                   
        #             if len(x_channels) >= 2 and len(y_channels) >= 2:    
        #                 for combo in tqdm(itertools.product(itertools.combinations(channel_locations_x, 2), itertools.combinations(channel_locations_y, 2))):
        #                     #flatten the combo
        #                     combo = [tuple(item) for sublist in combo for item in sublist]
        #                     combo = list(set(combo))
        #                     if len(combo) == 4:
        #                         combinations.append(combo)

        #     num_combinations = len(combinations)
        #     print(f'Number of valid combinations after matching x/y coords: {num_combinations}')
            
        #     # Filter combinations to only include those that contain the max_average_channel
        #     combinations = [combo for combo in combinations if max_average_channel in combo]

        #     # Generate combinations that include max_average_channel
        #     other_channels = [channel_locations[i] for i in range(len(channel_locations)) if i != max_average_channel]
        #     combinations = [
        #                     combo for combo in tqdm(itertools.combinations(other_channels, 4))#, total=num_combinations) 
        #                     if Polygon(combo).contains(Point(tuple(channel_locations[max_average_channel])))
        #                     ]            
        #     num_combinations = len(combinations)
        #     print(f'Number of valid combinations after max channel inclusion: {num_combinations}')

        #     # Sort combinations by area in descending order
        #     def area_of_combination(combination):
        #         polygon = Polygon(combination)
        #         return polygon.area
        #     combinations = sorted(combinations, key=area_of_combination, reverse=True)

        #     # Find the largest rectangle
        #     for combination in combinations:
        #         polygon = Polygon(combination)
        #         if is_rectangle(polygon):
        #             return combination  # Return the first rectangle found
            
        #     return None

        # def remove_channels_and_columns(merged_template, merging_channel_loc_tuples, rectangle_channels):
        #     # Remove the channels and columns that are not in rectangle_channels from merged_template and merging_channel_loc_tuples
        #     rectangle_indices = [merging_channel_loc_tuples.index(channel) for channel in rectangle_channels]
        #     merged_template = merged_template[:, rectangle_indices]
        #     merging_channel_loc_tuples = [merging_channel_loc_tuples[i] for i in rectangle_indices]
        #     return merged_template, merging_channel_loc_tuples

        # averages = calculate_column_averages(merged_template)

        # # Find max average column
        # max_average_column = find_max_average_column(averages)

        # # Find largest rectangle
        # rectangle_channels = find_largest_rectangle(merged_channel_loc, max_average_column)

        # # Remove channels and columns
        # merged_template, merging_channel_loc_tuples = remove_channels_and_columns(merged_template, merging_channel_loc_tuples, rectangle_channels) 

        # #TESTING A THEORY END

        template = [merged_template]
        # Iterate over the merged templates
        #Initialize list to store gtr objects
        gtrs = {}
        for i, template in enumerate(template):
            
            # Extract the sampling frequency for the current unit
            #hardcoded for now, fix later
            fs = 10000      

            transformed_template = np.array(template[0]).T
            trans_loc = merged_channel_loc[0]
            plot_dir = Path(recon_dir) / f'unit_{unit_id}' / 'axon_tracking_plots'
            key = 0
            
            # Generate amplitude_map figure
            try:
                fig_amp = plt.figure(figsize=(10, 5))
                ax_amp = fig_amp.add_subplot(111)
                #set_trace()
                ax_amp = plot_amplitude_map(transformed_template, 
                                            trans_loc, 
                                            log=False, 
                                            ax=ax_amp, 
                                            #cmap="PRGn", 
                                            colorbar=True, 
                                            colorbar_orientation="horizontal")
                ax_amp.set_title(f"Amplitude", fontsize=20)

                #Save figure
                fig_name = "amplitude_map.png"
                fig_path = plot_dir / fig_name
                if not os.path.exists(plot_dir):
                    os.makedirs(plot_dir)
                plt.savefig(fig_path, dpi=300, bbox_inches='tight')
                plt.close()

                #accept_units_amp_map.append(key)
            except Exception as e:
                #reject_units_amp_map.append(key)
                print(f"unit {key} failed to generate amplitude_map, error: {e}")

            # Generate peak latency map figure
            try:
                fig_peaks = plt.figure(figsize=(10, 5))
                ax_peaks = fig_peaks.add_subplot(111)
                ax_peaks = plot_peak_latency_map(transformed_template, merged_channel_loc[i], fs=fs, log=False, ax=ax_peaks, colorbar=True, colorbar_orientation="horizontal")
                ax_peaks.set_title(f"Peak latency", fontsize=20)

                #Save figure
                fig_name = "peak_latency_map.png"
                fig_path = plot_dir / fig_name
                plt.savefig(fig_path, dpi=300, bbox_inches='tight')
                plt.close()

                #accept_units_peak_latency_map.append(key)
            except Exception as e:
                #reject_units_peak_latency_map.append(key)
                print(f"unit {key} failed to generate peak_latency_map, error: {e}")

            # Select channels and generate selected channels figures
            def plot_selected_channels(fig_title, selected_channels, locations, filename):
                fig = plt.figure(figsize=(10, 5))
                ax = fig.add_subplot(111)
                ax.set_title(fig_title, fontsize=20)
                ax.plot(locations[:, 0], locations[:, 1], marker=".", color="grey", ls="", alpha=0.2)
                ax.plot(locations[selected_channels, 0], 
                        locations[selected_channels, 1], marker=".", color="k", ls="", alpha=0.5)
                ax.axis("off")
                fig_name = filename
                fig_path = plot_dir / fig_name
                plt.savefig(fig_path, dpi=300, bbox_inches='tight')
                plt.close()
            try:
                # Build axon tracking object
                verbose_bool = True
                gtr0 = GraphAxonTracking(transformed_template, merged_channel_loc[i], fs, verbose=verbose_bool, **params)
                

                # Select channels
                gtr0.select_channels()                

                # Use the function for each block
                plot_selected_channels(f"Selected after detection threshold: {gtr0._detect_threshold} ms", 
                                    np.array(list(gtr0._selected_channels_detect)), gtr0.locations, "detection_threshold.png")
                plot_selected_channels(f"Selected after peak std threshold: {gtr0._peak_std_threhsold} ms", 
                                        np.array(list(gtr0._selected_channels_peakstd)), gtr0.locations, "peak_std_threshold.png")
                plot_selected_channels(f"Selected after init delay threshold: {gtr0._init_delay} ms", 
                                        np.array(list(gtr0._selected_channels_init)), gtr0.locations, "init_delay_threshold.png")
                plot_selected_channels("Selected after all thresholds", gtr0.selected_channels, gtr0.locations, "all_thresholds.png")
                #accept_units_selected_channels.append(key)
            except Exception as e:
                #reject_units_selected_channels.append(key)
                print(f"unit {key} failed to generate selected_channels, error: {e}")              

            # Generate axon_reconstruction_heuristics figure
            try:
                gtr0.build_graph()
                gtr0.find_paths()
                gtr0._verbose = 1

                fig_graph = plt.figure(figsize=(10, 7))
                fig_graph = gtr0.plot_graph(node_search_labels=False, fig=fig_graph, cmap_nodes="viridis", cmap_edges="YlGn")

                fig_name = "axon_reconstruction_heuristics.png"
                fig_path = plot_dir / fig_name
                plt.savefig(fig_path, dpi=300, bbox_inches='tight')
                plt.close()

                #accept_units_axon_reconstruction_heuristics.append(key)
            except Exception as e:
                #reject_units_axon_reconstruction_heuristics.append(key)
                print(f"unit {key} failed to generate axon_reconstruction_heuristics, error: {e}")

            # Generate axon_reconstruction figure
            try:
                fig_graph = plt.figure(figsize=(12, 7))
                ax_raw = fig_graph.add_subplot(111)                
                axpaths_raw = ax_raw
                axpaths_raw = gtr0.plot_raw_branches(cmap="tab20", plot_bp=True, plot_neighbors=True, 
                                                    plot_full_template=True, ax=axpaths_raw)
                axpaths_raw.legend(fontsize=6)
                axpaths_raw.set_title("Raw Branches")

                fig_name = "axon_reconstruction.png"
                fig_path = plot_dir / fig_name
                plt.savefig(fig_path, dpi=600, bbox_inches='tight')
                plt.close()
                # gtrs[key] = compute_graph_propagation_velocity(template.T, merged_channel_loc[i], fs, 
                #                                         verbose=False, **params)

                #accept_units_axon_reconstruction.append(key)
                #flag = True
                #break
            except Exception as e:
                #reject_units_axon_reconstruction.append(key)
                print(f"unit {key} failed to generate axon_reconstruction, error: {e}")
                
            # Generate axon_reconstruction_velocities figure
            try:
                fvel, ax_vel = plt.subplots(figsize=(9, 15))
                ax_vel.axis("off")
                fvel = gtr0.plot_velocities(cmap="tab20", plot_outliers=True, fig=fvel, markersize=12, markersize_out=18, fs=20)
                plt.xticks(fontsize=16)
                plt.yticks(fontsize=16)

                fig_name = "axon_reconstruction_velocities.png"
                fig_path = plot_dir / fig_name
                plt.savefig(fig_path, dpi=600, bbox_inches='tight')
                plt.close()

                #accept_units_axon_reconstruction_velocities.append(key)
            except Exception as e:
                #reject_units_axon_reconstruction_velocities.append(key)
                print(f"unit {key} failed to generate axon_reconstruction_velocities, error: {e}")

2024-02-18 13:07:56,331 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details


Processing stream well001
Channel selection

Initial channels: 19390
12040 after detection filter
13468 after kurtosis filter
328 after peak std filter
18328 after init_delay filter
Removing isolated channels
Removed 68 isolated channels
Added 125 nodes
Added 218 edges
0 connected to init channel
Pruning
unit 0 failed to generate axon_reconstruction_heuristics, error: No branches found
unit 0 failed to generate axon_reconstruction, error: 'NoneType' object is not iterable
unit 0 failed to generate axon_reconstruction_velocities, error: 'NoneType' object is not iterable


KeyboardInterrupt: 

In [ ]:
recordings_dir = './AxonReconPipeline/data/temp_data/recordings'
recordings_dir = os.path.abspath(recordings_dir)
recordings_dir = str(recordings_dir)

In [ ]:
##3. Extract Templates
#slighlty modified version of extract_templates_hp from main.py
def extract_templates_hp(continuous_h5_dirs, 
                         sorting_lists_by_scan,
                         allowed_scan_types = ["AxonTracking"], 
                         templates_dir='./AxonReconPipeline/data/temp_data/templates'):
    te_params = dict()
    te_params['align_cutout'] = True #Align waveforms by max waveform peak
    te_params['upsample'] = 2 #Factor by which to upsample waveforms
    te_params['rm_outliers'] = True #Check if outliers should be removed
    te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
    te_params['n_neighbors'] = 10 #Number of neighbors for outlier detection
    te_params['peak_cutout'] = 2 #Looking for peak +- this value around the expected peak (removing minor offsets)
    te_params['overwrite_wf'] = False #Flag if waveform extraction should be repeated (e.g. different cutouts)
    te_params['overwrite_tmp'] = True #Flag if templates should be recalculated if already existing

    qc_params = dict()
    qc_params['min_n_spikes'] = 500 #Minimum number of spikes to be detected for a unit for template extraction to take place
    qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort

    for dir in continuous_h5_dirs.values():
        h5_file_path = dir['h5_file_path']
        sorting_list_by_stream = sorting_lists_by_scan[0] #for debugging
        scantype = dir['scanType']
        if scantype in allowed_scan_types:
        #logger.info(f"Processing {h5_file_path}...")
            #te_hp.extract_templates_from_sorting_dict(sorting_list_by_stream, h5_file_path, qc_params, te_params)
            #spreading out functions in laters cells for debugging, just return varibales needed for extract_templates_from_sorting_dict forl ater
            return sorting_list_by_stream, h5_file_path, qc_params, te_params


#Extract recording details from the .h5 file path
templates_dir='./AxonReconPipeline/data/temp_data/templates'
#templates_by_scan = extract_templates_hp(continuous_h5_dirs, 
sorting_list_by_stream, h5_file_path, qc_params, te_params = extract_templates_hp(continuous_h5_dirs,
                                                                                sorting_lists_by_scan, 
                                                                                allowed_scan_types = allowed_scan_types,
                                                                                templates_dir=templates_dir)

In [ ]:
import os, h5py
import spikeinterface.full as si
import numpy as np
import scipy as sp
import sklearn as sk
from tqdm import tqdm
import shutil

#from axon_tracking import spike_sorting as ss
import spike_sorting as ss
from template_extraction import select_units
#Slightly modified version of extract_templates_from_sorting_dict from template_extraction.py
def extract_templates_from_sorting_dict(sorting_dict, h5_file_path, qc_params={}, te_params={}):
    #rec_list = list(sorting_dict.keys())
    rec_list = [h5_file_path]    

    for rec_path in rec_list:
        #sorting_list = sorting_dict[rec_path]
        sorting_list = sorting_dict
        #sorting_list.sort()

        for sorting in sorting_list:
        #for sorting_path in sorting_list:
            #sorting = si.KiloSortSortingExtractor(sorting_path)
            sorting_path = sorting._kwargs['folder_path']
            #Temporary debug
            #if 'Network' in stream_id, replace with 'AxonTracking'
            if 'Network' in sorting_path:
                sorting_path = sorting_path.replace('Network', 'AxonTracking')
            stream_id = [p for p in sorting_path.split('/') if p.startswith('well')][0] #Find out which well this belongs to
            
            print(stream_id)
            #debug:
            stream_number = int(stream_id[-1])
            if stream_number > 0:
                continue
            #debug end
            
            #rec_names, common_el, pos = ss.find_common_electrodes(rec_path, stream_id)
            multirecording, pos = ss.concatenate_recording_slices(rec_path, stream_id)

            #duration = int(h5['assay']['inputs']['record_time'][0].decode('UTF-8')) * n_recs #In case we want to use firing rate as criterion
            cleaned_sorting = select_units(sorting, **qc_params)
            cleaned_sorting = si.remove_excess_spikes(cleaned_sorting, multirecording) #Relevant if last spike time == recording_length
            cleaned_sorting.register_recording(multirecording)
            segment_sorting = si.SplitSegmentSorting(cleaned_sorting, multirecording)
            return stream_id, segment_sorting, sorting_path, pos, te_params
        
stream_id, segment_sorting, sorting_path, pos, te_params = extract_templates_from_sorting_dict(sorting_list_by_stream, h5_file_path, qc_params, te_params)
save_root = sorting_path


In [ ]:
sel_unit_ids = segment_sorting.get_unit_ids()
template_save_path = os.path.join(save_root, 'templates')
if not os.path.exists(template_save_path):
    os.makedirs(template_save_path)
    
for sel_unit_id in tqdm(sel_unit_ids):
    sel_unit_id= sel_unit_id
    #choosing a random unit
    #sel_unit_id = 29
    sel_unit_id = 345
    break


In [ ]:
from template_extraction import extract_waveforms
#def combine_templates(stream_id, segment_sorting, sel_unit_id, save_root, peak_cutout=2, align_cutout=True, upsample=2, rm_outliers=True, 
n_jobs=16
#, 
#n_neighbors=10, 
overwrite_wf=False
#,
#overwrite_tmp = True
#):

#full_path = segment_sorting._kwargs['recording_list'][0]._parent_recording._kwargs['recording']._kwargs['file_path']
full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._kwargs['parent_recording']._kwargs['recording']._kwargs['file_path']
# cutout = get_assay_information(full_path)
# if align_cutout:
#     wf_length = np.int16((sum(cutout) - 2*peak_cutout) * upsample) #length of waveforms after adjusting for potential peak alignments
#     template_matrix = np.full([wf_length, 26400], np.nan)
# else:
#     wf_length = np.int16(sum(cutout) * upsample)
#     template_matrix = np.full([wf_length, 26400], np.nan)


In [ ]:
    
extract_waveforms(segment_sorting, stream_id, save_root, n_jobs, overwrite_wf)

In [ ]:

h5 = h5py.File(full_path)
rec_names = list(h5['wells'][stream_id].keys())
#Find a way to average common electrodes
# Initialize count_matrix with the same shape as template_matrix, filled with zeros
#count_matrix = np.zeros(template_matrix.shape)
template_list = []
channel_locations_list = []  
    
aw = True
if aw:
    for sel_idx, rec_name in enumerate(rec_names): 
        rec = si.MaxwellRecordingExtractor(full_path,stream_id=stream_id,rec_name=rec_name)
        els = rec.get_property("contact_vector")["electrode"]
        seg_sort = si.SelectSegmentSorting(segment_sorting, sel_idx)
        seg_we = si.load_waveforms(os.path.join(save_root, 'waveforms', 'seg' + str(sel_idx)), sorting = seg_sort)
        template = seg_we.get_template(sel_unit_id)
        channel_locations = seg_we.get_channel_locations()
        channel_locations_list.append(channel_locations)
        template_list.append(template)
    
    # merged_template, merged_channel_loc = ra.merge_templates(template_list, 
    #                                                         channel_locations_list, 
    #                                                         plot_dir = save_root+'/merged_templates')
    # return merged_template  

# template_save_file = os.path.join(template_save_path, str(sel_unit_id) + '.npy')

# if not os.path.isfile(template_save_file) or te_params['overwrite_tmp']:
#     try:
#         merged_template = combine_templates(stream_id, segment_sorting, sel_unit_id, save_root, **te_params)
#         #grid = convert_to_grid(template_matrix, pos)
#         np.save(template_save_file, merged_template)
#     except Exception as e:
#         print(f'Unit {sel_unit_id} encountered the following error:\n {e}')

In [ ]:
### Initiate Axon trace algo
import numpy as np
import matplotlib.pylab as plt
import MEAutility as mu
from scipy.signal import resample_poly
from scipy.stats import kurtosis, linregress
from matplotlib import gridspec
from scipy import io
import numpy as np
import networkx as nx
from pathlib import Path
import sys
from pprint import pprint
from axon_velocity import *
from axon_velocity.models import load_cell
from axon_velocity.evaluation import *
import os
from pathlib import Path
import pickle
import re
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from probeinterface import plotting as plotting_probe
import json

import concurrent.futures
from tqdm import tqdm
from multiprocessing import Manager

def process_channel(args):
    i, k, j, channel = args[:4]
    channel_to_index,merging_template, merging_count_at_channel_by_sample = args[4:7]
    templates, merging_channel_loc_tuples = args[7:]

    # channel j
    #channel = channel_loc_tuples[j]
    found_idx = channel_to_index.get(channel)

    if found_idx is not None:
        # Found match, average values  
        merged_sample_at_channel = merging_template[k][found_idx]
        sample_to_be_merged = templates[i][k][j]

        # Choose to avg or replace based merge_count_at channel
        merge_count = merging_count_at_channel_by_sample[k][found_idx]

        if merge_count > 0:
            # Weighted Average - probably need to improve this method later
            total_samples_at_channel = merge_count + 1
            merged_sample_at_channel = (merged_sample_at_channel*merge_count + sample_to_be_merged) / (total_samples_at_channel)
        elif merge_count <= 0 and merged_sample_at_channel == 0:
            merged_sample_at_channel = sample_to_be_merged
        else:
            print("Something funky is going on")

        merging_template[k][found_idx] = merged_sample_at_channel
        merging_count_at_channel_by_sample[k][found_idx] += 1
    else:
        # No match, append
        sample_to_append = templates[i][k][j]
        #channel_loc_to_append = channel_loc[j] 

        # Extend merged_templates
        merging_template[k] = list(merging_template[k]) + [sample_to_append]

        # Extend merged_count_at_channel_by_sample, set to 1 representing 1 value at channel
        merging_count_at_channel_by_sample[k] = list(merging_count_at_channel_by_sample[k]) + [1]

        # Extend merged_channel_loc
        merging_channel_loc_tuples.append(channel)

    return merging_template, merging_channel_loc_tuples

def merge_templates(templates, channel_locations, plot_dir):
    ###Merge
    merged_template = []
    merged_channel_loc = []
    merging_template = None
    merging_count_at_channel_by_sample = None
    merging_channel_loc_tuples = None
    
    for i in tqdm(range(len(templates)), desc='Merging templates'):
        #template key
        #rec_num = rec_nums[key]
        #are_keys_sequential(channel_locations)
        channel_loc = channel_locations[i]
        
        if len(merged_template) == 0:
            # First template, so just copy over
            merged_template.append(templates[i]) 
            merged_channel_loc.append(channel_loc)

            #prepare array to count how many times samples at a channel are merged
            num_samples = merged_template[-1][0].shape[0]
            # Calculate new size, Create pre-sized merged_template array
            merged_count_at_channel_by_sample = [np.array([np.zeros(num_samples, dtype='float32')] * len(merged_template[-1]), dtype='float32')]
            new_merged_count_at_channel_by_sample = [np.array([np.zeros(num_samples, dtype='float32')] * len(merged_template[-1]), dtype='float32')]
            #Add 1 indicating that first template represents 1 legit value for ready to be merged 
            new_merged_count_at_channel_by_sample[-1] += 1
            merged_count_at_channel_by_sample = new_merged_count_at_channel_by_sample 
        else:
            # Not first template, need to merge
            new_template = []
            new_channel_loc = [] 
            
            # Convert numpy arrays to tuples
            channel_loc_tuples = [tuple(channel) for channel in channel_loc]
            # make temporary list of lists template for merging
            if merging_template is None:
                merging_channel_loc_tuples = [tuple(channel) for channel in merged_channel_loc[-1]]
                merging_template = [arr.tolist() for arr in merged_template[-1]]
                merging_count_at_channel_by_sample = [arr.tolist() for arr in merged_count_at_channel_by_sample[-1]]            

            # Create a dictionary mapping channels to their indices
            channel_to_index = {channel: idx for idx, channel in enumerate(merging_channel_loc_tuples)}
            
            #merging_channel_loc = merged_channel_loc[-1]

            for k in range(len(templates[i])):                
                # sample k
                for j, channel in enumerate(channel_loc_tuples):
                    # channel j
                    found_idx = channel_to_index.get(channel)

                    if found_idx is not None:
                        # Found match, average values  
                        try:
                            merged_sample_at_channel = merging_template[k][found_idx]
                            sample_to_be_merged = templates[i][k][j]
                        except:
                            print("Error")

                        # Choose to avg or replace based merge_count_at channel
                        try: merge_count = merging_count_at_channel_by_sample[k][found_idx]
                        except:
                            print("Error")

                        if merge_count > 0:
                            # Weighted Average - probably need to improve this method later
                            total_samples_at_channel = merge_count + 1
                            merged_sample_at_channel = (merged_sample_at_channel*merge_count + sample_to_be_merged) / (total_samples_at_channel)
                        elif merge_count <= 0 and merged_sample_at_channel == 0:
                            merged_sample_at_channel = sample_to_be_merged
                        else:
                            print("Something funky is going on")

                        merging_template[k][found_idx] = merged_sample_at_channel
                        merging_count_at_channel_by_sample[k][found_idx] += 1
                    else:
                        # No match, append
                        sample_to_append = templates[i][k][j]
                        channel_loc_to_append = channel_loc[j] 

                        # Extend merged_templates
                        merging_template[k] = list(merging_template[k]) + [sample_to_append]

                        # Extend merged_count_at_channel_by_sample, set to 1 representing 1 value at channel
                        merging_count_at_channel_by_sample[k] = list(merging_count_at_channel_by_sample[k]) + [1]

                        # Extend merged_channel_loc
                        merging_channel_loc_tuples.append(channel)

                        #Keep lists consistent length
                        # Find the maximum length of any list in merging_template
                        max_length = max(len(lst) for lst in merging_template)

                        # Extend all lists in merging_template to the maximum length
                        for lst in merging_template:
                            lst.extend([0] * (max_length - len(lst)))

                        # Do the same for merging_count_at_channel_by_sample
                        for lst in merging_count_at_channel_by_sample:
                            lst.extend([0] * (max_length - len(lst)))

                        # Update dictionary mapping channels to their indices
                        channel_to_index = {chan: idx for idx, chan in enumerate(merging_channel_loc_tuples)}                     
    
    #Validation
    assert len(merged_template[-1]) == len(merging_template)
    for i in range(len(merging_template)):
        assert len(merging_template[i]) <= 26400
    assert len(merging_channel_loc_tuples) <= 26400          
    
    # Convert channel_loc to float64 and make it a numpy array of numpy arrays
    new_merged_channel_loc = np.array([np.array(tuple, dtype='float64') for tuple in merging_channel_loc_tuples])
    merged_channel_loc[-1] = new_merged_channel_loc   

    # Convert template to float32 and make it a numpy array of numpy arrays
    new_merged_template = np.array([np.array(template, dtype='float32') for template in merging_template])
    merged_template[-1] = np.array([np.array(template, dtype='float32') for template in merging_template])

    return merged_template, merged_channel_loc

merged_template, merged_channel_loc = merge_templates(template_list, 
                                                         channel_locations_list, 
                                                         plot_dir = save_root+'/merged_templates')


In [ ]:
from IPython.core.debugger import set_trace
from pathlib import Path

#Define Algorithm Params
params = get_default_graph_velocity_params()
# change params (fig6 Buccino axon_velocity paper example)
params['detect_threshold'] = 0.01
params['kurt_threshold'] = 0.1
params['peak_std_threshold'] = 0.8
params['upsample'] = 5
params['neighbor_radius'] = 100
params['r2_threshold'] = 0.8

# AW invention. I dont remember why I did this. 4Jan2023
params["theilsen_maxiter"] = 10000
#pprint(params)
#
tempaltes = [merged_template]
# Iterate over the merged templates
#Initialize list to store gtr objects
gtrs = {}
for i, template in enumerate(tempaltes):
    
    # Extract the sampling frequency for the current unit
    fs = 10000      

    transformed_template = np.array(template[0]).T
    trans_loc = merged_channel_loc[0]
    plot_dir = Path(save_root) / 'axon_tracking_plots'
    key = 0
    
    # Generate amplitude_map figure
    try:
        fig_amp = plt.figure(figsize=(10, 5))
        ax_amp = fig_amp.add_subplot(111)
        #set_trace()
        ax_amp = plot_amplitude_map(transformed_template, trans_loc, 
                                    log=True, ax=ax_amp, 
                                    #cmap="PRGn", 
                                    colorbar=True, 
                                    colorbar_orientation="horizontal")
        ax_amp.set_title(f"Amplitude", fontsize=20)

        #Save figure
        fig_name = "amplitude_map.png"
        fig_path = plot_dir / fig_name
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)
        plt.savefig(fig_path, dpi=300, bbox_inches='tight')
        plt.close()

        #accept_units_amp_map.append(key)
    except Exception as e:
        #reject_units_amp_map.append(key)
        print(f"unit {key} failed to generate amplitude_map, error: {e}")

    # Generate peak latency map figure
    try:
        fig_peaks = plt.figure(figsize=(10, 5))
        ax_peaks = fig_peaks.add_subplot(111)
        ax_peaks = plot_peak_latency_map(transformed_template, merged_channel_loc[i], fs=fs, log=False, ax=ax_peaks, colorbar=True, colorbar_orientation="horizontal")
        ax_peaks.set_title(f"Peak latency", fontsize=20)

        #Save figure
        fig_name = "peak_latency_map.png"
        fig_path = plot_dir / fig_name
        plt.savefig(fig_path, dpi=300, bbox_inches='tight')
        plt.close()

        #accept_units_peak_latency_map.append(key)
    except Exception as e:
        #reject_units_peak_latency_map.append(key)
        print(f"unit {key} failed to generate peak_latency_map, error: {e}")

    # Select channels and generate selected channels figures
    def plot_selected_channels(fig_title, selected_channels, locations, filename):
        fig = plt.figure(figsize=(10, 5))
        ax = fig.add_subplot(111)
        ax.set_title(fig_title, fontsize=20)
        ax.plot(locations[:, 0], locations[:, 1], marker=".", color="grey", ls="", alpha=0.2)
        ax.plot(locations[selected_channels, 0], 
                locations[selected_channels, 1], marker=".", color="k", ls="", alpha=0.5)
        ax.axis("off")
        fig_name = filename
        fig_path = plot_dir / fig_name
        plt.savefig(fig_path, dpi=300, bbox_inches='tight')
        plt.close()
    try:
        # Build axon tracking object
        verbose_bool = True
        gtr0 = GraphAxonTracking(transformed_template, merged_channel_loc[i], fs, verbose=verbose_bool, **params)
        

        # Select channels
        gtr0.select_channels()                

        # Use the function for each block
        plot_selected_channels(f"Selected after detection threshold: {gtr0._detect_threshold} ms", 
                            np.array(list(gtr0._selected_channels_detect)), gtr0.locations, "detection_threshold.png")
        plot_selected_channels(f"Selected after peak std threshold: {gtr0._peak_std_threhsold} ms", 
                                np.array(list(gtr0._selected_channels_peakstd)), gtr0.locations, "peak_std_threshold.png")
        plot_selected_channels(f"Selected after init delay threshold: {gtr0._init_delay} ms", 
                                np.array(list(gtr0._selected_channels_init)), gtr0.locations, "init_delay_threshold.png")
        plot_selected_channels("Selected after all thresholds", gtr0.selected_channels, gtr0.locations, "all_thresholds.png")
        #accept_units_selected_channels.append(key)
    except Exception as e:
        #reject_units_selected_channels.append(key)
        print(f"unit {key} failed to generate selected_channels, error: {e}")              

    # Generate axon_reconstruction_heuristics figure
    try:
        gtr0.build_graph()
        gtr0.find_paths()
        gtr0._verbose = 1

        fig_graph = plt.figure(figsize=(10, 7))
        fig_graph = gtr0.plot_graph(node_search_labels=False, fig=fig_graph, cmap_nodes="viridis", cmap_edges="YlGn")

        fig_name = "axon_reconstruction_heuristics.png"
        fig_path = plot_dir / fig_name
        plt.savefig(fig_path, dpi=300, bbox_inches='tight')
        plt.close()

        #accept_units_axon_reconstruction_heuristics.append(key)
    except Exception as e:
        #reject_units_axon_reconstruction_heuristics.append(key)
        print(f"unit {key} failed to generate axon_reconstruction_heuristics, error: {e}")

    # Generate axon_reconstruction figure
    try:
        fig_graph = plt.figure(figsize=(12, 7))
        ax_raw = fig_graph.add_subplot(111)                
        axpaths_raw = ax_raw
        axpaths_raw = gtr0.plot_raw_branches(cmap="tab20", plot_bp=True, plot_neighbors=True, 
                                            plot_full_template=True, ax=axpaths_raw)
        axpaths_raw.legend(fontsize=6)
        axpaths_raw.set_title("Raw Branches")

        fig_name = "axon_reconstruction.png"
        fig_path = plot_dir / fig_name
        plt.savefig(fig_path, dpi=600, bbox_inches='tight')
        plt.close()
        gtrs[key] = compute_graph_propagation_velocity(template.T, merged_channel_loc[i], fs, 
                                                verbose=False, **params)

        #accept_units_axon_reconstruction.append(key)
        #flag = True
        #break
    except Exception as e:
        #reject_units_axon_reconstruction.append(key)
        print(f"unit {key} failed to generate axon_reconstruction, error: {e}")
        
    # Generate axon_reconstruction_velocities figure
    try:
        fvel, ax_vel = plt.subplots(figsize=(9, 15))
        ax_vel.axis("off")
        fvel = gtr0.plot_velocities(cmap="tab20", plot_outliers=True, fig=fvel, markersize=12, markersize_out=18, fs=20)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)

        fig_name = "axon_reconstruction_velocities.png"
        fig_path = plot_dir / fig_name
        plt.savefig(fig_path, dpi=600, bbox_inches='tight')
        plt.close()

        #accept_units_axon_reconstruction_velocities.append(key)
    except Exception as e:
        #reject_units_axon_reconstruction_velocities.append(key)
        print(f"unit {key} failed to generate axon_reconstruction_velocities, error: {e}")
        
# # Print the number of accepted and rejected units for each figure
# print(f"Amplitude map: {len(accept_units_amp_map)} accepted, {len(reject_units_amp_map)} rejected")
# print(f"Peak latency map: {len(accept_units_peak_latency_map)} accepted, {len(reject_units_peak_latency_map)} rejected")
# print(f"Selected channels: {len(accept_units_selected_channels)} accepted, {len(reject_units_selected_channels)} rejected")
# print(f"Axon reconstruction heuristics: {len(accept_units_axon_reconstruction_heuristics)} accepted, {len(reject_units_axon_reconstruction_heuristics)} rejected")
# print(f"Axon reconstruction: {len(accept_units_axon_reconstruction)} accepted, {len(reject_units_axon_reconstruction)} rejected")
# print(f"Axon reconstruction velocities: {len(accept_units_axon_reconstruction_velocities)} accepted, {len(reject_units_axon_reconstruction_velocities)} rejected")


In [ ]:
# Check if they have the same shape
if new_merged_channel_loc.shape == merged_channel_loc[-1].shape:
    # Check if they have the same dtype
    if new_merged_channel_loc.dtype == merged_channel_loc[-1].dtype:
        merged_channel_loc[-1] = new_merged_channel_loc
    else:
        print("new_merged_channel_loc and merged_channel_loc[-1] have different data types")
else:
    print("new_merged_channel_loc and merged_channel_loc[-1] have different shapes")

In [ ]:
args = (j, channel, i, k, channel_to_index, merging_template, merging_count_at_channel_by_sample, templates, channel_to_index, merging_channel_loc_tuples)

In [ ]:
executor.map(process_channel, [(j, channel) + args for j, channel in enumerate(channel_loc_tuples)])

In [ ]:
def process_channel(args):
    i, k, j, channel, channel_lockchannel_to_index = args[:5]
    merging_template, merging_count_at_channel_by_sample = args[5:7]
    templates, channel_to_index, merging_channel_loc_tuples = args[7:]

In [ ]:
# make temporary list of lists template for merging
merging_template = [arr.tolist() for arr in merged_templates[-1]]

In [ ]:
else:
    # No match, append
    sample_to_append = templates[i][k][j]
    channel_loc_to_append = channel_loc[j] 

    # Extend merged_templates
    merged_templates[-1][k] = list(merged_templates[-1][k]) + [sample_to_append]

    # Extend merged_count_at_channel_by_sample
    merged_count_at_channel_by_sample[-1][k] = list(merged_count_at_channel_by_sample[-1][k]) + [1]

    # Extend merged_channel_loc
    merged_channel_loc[-1] = list(merged_channel_loc[-1]) + [channel_loc_to_append]

In [ ]:
# No match, append
sample_to_append = templates[i][k][j]
channel_loc_to_append = channel_loc[j] 

# Extend merged_templates
merged_templates[-1] = np.concatenate((merged_templates[-1], [sample_to_append]))

# Extend merged_count_at_channel_by_sample
merged_count_at_channel_by_sample[-1] = np.concatenate((merged_count_at_channel_by_sample[-1], [1]))

# Extend merged_channel_loc
merged_channel_loc[-1] = np.concatenate((merged_channel_loc[-1], [channel_loc_to_append]), axis=0)

In [ ]:
# Convert numpy arrays to tuples
channel_loc_tuples = [tuple(channel) for channel in channel_loc]
merged_channel_loc_tuples = [tuple(channel) for channel in merged_channel_loc[-1]]

# Get common channels
common_channels_tuples = set(channel_loc_tuples).intersection(set(merged_channel_loc_tuples))

# Get indices of common channels in channel_loc
common_indices = [idx for idx, channel in enumerate(channel_loc_tuples) if channel in common_channels_tuples]

In [ ]:
# Convert numpy arrays to tuples
channel_loc_tuples = [tuple(channel) for channel in channel_loc]
merged_channel_loc_tuples = [tuple(channel) for channel in merged_channel_loc[-1]]

# Get common channels
common_channels_tuples = set(channel_loc_tuples).intersection(set(merged_channel_loc_tuples))

# Get indices of common channels in channel_loc
common_indices = [idx for idx, channel in enumerate(channel_loc_tuples) if channel in common_channels_tuples]

In [ ]:
# Add 1 indicating that first template represents 1 legit value for ready to be merged 
new_merged_count_at_channel_by_sample[-1] += 1